In [1]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from timeit import default_timer as timer
import datetime
import numpy as np
from utils.config import PATH_SCENARIOS_REDUCED, PATH_DISTANCES, PATH_SCENARIO_PROBABILITY, PATH_INITIAL_ALLOCATION
from utils.stochastic_program import StochasticProgramFactory, StochasticProgram
from collections import defaultdict

### Prepare scenario data
Make scenarios complete cross product

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [5]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [6]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [7]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [8]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [9]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Get Model Parameters


In [10]:
demand = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

In [11]:
start = timer()

for _, row in scenarios.reset_index().iterrows():
    demand[row.start_hex_ids][row.end_hex_ids][row.time.hour][row.vehicle_types][row.scenarios] = row.demand

end = timer()
print(f"Successfully converted dataframe into dictionary in {(end - start):.2f} seconds")

Successfully converted dataframe into dictionary in 5.65 seconds


In [12]:
distances = pd.read_pickle(PATH_DISTANCES)

In [13]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [14]:
start = timer()

vehicle_types = list(scenarios.reset_index()['vehicle_types'].unique())

for _, row in distances.reset_index().iterrows():
    for vehicle_type in vehicle_types:
        cost[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"cost_{vehicle_type}"]
        profit[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"profit_{vehicle_type}"]

end = timer()

In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [17]:
initial_allocation = pd.read_pickle(PATH_INITIAL_ALLOCATION)

In [18]:
factory = StochasticProgramFactory(scenarios, distances, probabilities, initial_allocation, node_df)

In [19]:
factory.convert_parameters()

convert_probabilities finished in 0.00 seconds
convert_distances finished in 0.08 seconds
convert_demand finished in 6.05 seconds
convert_nodes finished in 0.01 seconds
convert_parameters finished in 6.14 seconds


In [48]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.01 seconds


In [49]:
stochastic_program.create_model()

create_variables finished in 0.77 seconds
create_objective_function finished in 1.45 seconds
create_demand_constraints finished in 1.18 seconds
create_big_u_sum_constraints finished in 0.35 seconds
create_relocation_binary_constraints finished in 0.04 seconds
create_unfulfilled_demand_binary_constraints finished in 0.04 seconds
create_max_trips_constraints finished in 0.45 seconds
create_vehicle_movement_constraints finished in 0.49 seconds
create_initial_allocation_constraints finished in 0.00 seconds
create_non_anticipativity_constraints finished in 0.70 seconds
create_constraints finished in 3.26 seconds
create_model finished in 6.35 seconds


In [53]:
stochastic_program.model.solve()

1

In [65]:
from pulp import LpStatus, value
import pulp

In [66]:
stochastic_program.model.status
print("Status:", LpStatus[stochastic_program.model.status])


Status: Optimal


In [67]:
model = stochastic_program.model

In [68]:
value(model.objective)

9588.590936837023

In [70]:
for v in model.variables():
    print(v.name, "=", v.value)
    break

[autoreload of utils.stochastic_program failed: Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\envs\VEHICLE_REPOSITION_SCI\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Miniconda3\envs\VEHICLE_REPOSITION_SCI\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\ProgramData\Miniconda3\envs\VEHICLE_REPOSITION_SCI\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\ProgramData\Miniconda3\envs\VEHICLE_REPOSITION_SCI\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<froz

In [ ]:
model.variables()[0]